In [45]:
import csv

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

# Specify each path

In [46]:
dataset = 'model/keypoint_classifier/keypoint.csv'
model_save_path = 'model/keypoint_classifier/keypoint_classifier.hdf5'
tflite_save_path = 'model/keypoint_classifier/keypoint_classifier.tflite'

# Set number of classes

In [47]:
NUM_CLASSES = 7

# Dataset reading

In [48]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))

In [49]:
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED)

# Model building

In [51]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [52]:
model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout_4 (Dropout)         (None, 42)                0         
                                                                 
 dense_6 (Dense)             (None, 20)                860       
                                                                 
 dropout_5 (Dropout)         (None, 20)                0         
                                                                 
 dense_7 (Dense)             (None, 10)                210       
                                                                 
 dense_8 (Dense)             (None, 7)                 77        
                                                                 
Total params: 1147 (4.48 KB)
Trainable params: 1147 (4.48 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [53]:
# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [54]:
# Model compilation
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Model training

In [55]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
 1/15 [=>............................] - ETA: 13s - loss: 1.8525 - accuracy: 0.2500
Epoch 1: saving model to model/keypoint_classifier\keypoint_classifier.hdf5
15/15 [==============================] - 1s 21ms/step - loss: 1.8241 - accuracy: 0.2430 - val_loss: 1.7256 - val_accuracy: 0.2860
Epoch 2/1000
 1/15 [=>............................] - ETA: 0s - loss: 1.7674 - accuracy: 0.2578
Epoch 2: saving model to model/keypoint_classifier\keypoint_classifier.hdf5
15/15 [==============================] - 0s 6ms/step - loss: 1.7429 - accuracy: 0.2882 - val_loss: 1.6338 - val_accuracy: 0.3653
Epoch 3/1000
 1/15 [=>............................] - ETA: 0s - loss: 1.7294 - accuracy: 0.2734
Epoch 3: saving model to model/keypoint_classifier\keypoint_classifier.hdf5
15/15 [==============================] - 0s 6ms/step - loss: 1.6643 - accuracy: 0.3460 - val_loss: 1.5625 - val_accuracy: 0.4264


c:\Users\Aadi\OneDrive\Desktop\TFODCourse\tfod\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 4/1000
 1/15 [=>............................] - ETA: 0s - loss: 1.6739 - accuracy: 0.3750
Epoch 4: saving model to model/keypoint_classifier\keypoint_classifier.hdf5
15/15 [==============================] - 0s 6ms/step - loss: 1.6264 - accuracy: 0.3785 - val_loss: 1.5002 - val_accuracy: 0.4595
Epoch 5/1000
 1/15 [=>............................] - ETA: 0s - loss: 1.5423 - accuracy: 0.4453
Epoch 5: saving model to model/keypoint_classifier\keypoint_classifier.hdf5
15/15 [==============================] - 0s 6ms/step - loss: 1.5752 - accuracy: 0.3917 - val_loss: 1.4497 - val_accuracy: 0.4810
Epoch 6/1000
 1/15 [=>............................] - ETA: 0s - loss: 1.6708 - accuracy: 0.3438
Epoch 6: saving model to model/keypoint_classifier\keypoint_classifier.hdf5
15/15 [==============================] - 0s 6ms/step - loss: 1.5364 - accuracy: 0.4028 - val_loss: 1.4011 - val_accuracy: 0.4876
Epoch 7/1000
 1/15 [=>............................] - ETA: 0s - loss: 1.6349 - accuracy: 0.3125
E

In [56]:
# Model evaluation
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

5/5 [==============================] - 0s 2ms/step - loss: 0.3268 - accuracy: 0.9107


In [57]:
# Loading the saved model
model = tf.keras.models.load_model(model_save_path)

In [58]:
# Inference test
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

1/1 [==============================] - 0s 89ms/step
[2.2252298e-05 6.8087443e-03 9.7650313e-01 1.7787281e-08 1.4042548e-02
 2.6233771e-03 5.2144933e-10]
2


# Confusion matrix

In [59]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

def print_confusion_matrix(y_true, y_pred, report=True):
    labels = sorted(list(set(y_true)))
    cmx_data = confusion_matrix(y_true, y_pred, labels=labels)
    
    df_cmx = pd.DataFrame(cmx_data, index=labels, columns=labels)
 
    fig, ax = plt.subplots(figsize=(7, 6))
    sns.heatmap(df_cmx, annot=True, fmt='g' ,square=False)
    ax.set_ylim(len(set(y_true)), 0)
    plt.show()
    
    if report:
        print('Classification Report')
        print(classification_report(y_test, y_pred))

Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)

print_confusion_matrix(y_test, y_pred)

ImportError: Unable to import required dependencies:
pytz: No module named 'pytz'

# Convert to model for Tensorflow-Lite

In [60]:
# Save as a model dedicated to inference
model.save(model_save_path, include_optimizer=False)

In [61]:
# Transform model (quantization)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

INFO:tensorflow:Assets written to: C:\Users\Aadi\AppData\Local\Temp\tmp45afny1h\assets


INFO:tensorflow:Assets written to: C:\Users\Aadi\AppData\Local\Temp\tmp45afny1h\assets


6704

# Inference test

In [62]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

In [63]:
# Get I / O tensor
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [64]:
interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]]))

In [65]:
%%time
# Inference implementation
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

CPU times: total: 0 ns
Wall time: 0 ns


In [66]:
print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))

[2.2252316e-05 6.8087406e-03 9.7650313e-01 1.7787247e-08 1.4042540e-02
 2.6233757e-03 5.2144827e-10]
2
